<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla, Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias, Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Luis Figueroa
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `https://github.com/LuisFigueroaG/MDS7202-Laboratorio-de-Programacion-Cientifica-para-Ciencia-de-Datos`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [108]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [109]:
df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Ejemplo de Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [110]:
def custom_features(dataframe_in):
    dataframe_in['InvoiceDate'] = pd.to_datetime(dataframe_in['InvoiceDate'])

    df_features = dataframe_in.groupby('Customer ID').agg(
        Length=('InvoiceDate', lambda x: (x.max() - x.min()).days),
        Recency=('InvoiceDate', lambda x: (datetime.datetime.now() - x.max()).days),
        Monetary=('Price', lambda x: (x * dataframe_in.loc[x.index, 'Quantity']).mean()),
        Frequency=('Invoice', 'count'),
        Periodicity=('InvoiceDate', lambda x: x.diff().dt.days.std())
    ).reset_index()

    # Reemplazar los valores nulos de Periodicity con 0
    df_features['Periodicity'].fillna(0, inplace=True)

    return df_features

In [111]:
df_features = custom_features(df_retail)
df_features.head(5)

,Customer ID,Length,Recency,Monetary,Frequency,Periodicity
0,12346.0,196,4731,11.298788,33,21.724076
1,12347.0,37,4569,18.638310,71,4.422346
2,12348.0,0,4640,11.108000,20,0.000000
3,12349.0,181,4609,26.187647,102,16.200990
4,12351.0,0,4577,14.330000,21,0.000000


## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [112]:
class MinMax(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.min_vals = {}
        self.max_vals = {}

    def fit(self, X, y=None):
        for col in X.columns:
            self.min_vals[col] = X[col].min()
            self.max_vals[col] = X[col].max()
        return self

    def transform(self, X):
        X_transformed = X.copy()
        for col in X.columns:
            X_transformed[col] = (X[col] - self.min_vals[col]) / (self.max_vals[col] - self.min_vals[col])
        return X_transformed


### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [113]:

# Configura el primer paso del pipeline para obtener características LRMFP
step1 = ('custom_features', FunctionTransformer(custom_features))

# Paso 2: Aplicar MinMax scaler a todas las columnas generadas en el paso 1
preprocessor = ColumnTransformer(
    transformers=[('minmax', MinMax(), ['Length', 'Recency', 'Monetary', 'Frequency','Periodicity'])],
    remainder='drop')

# Paso 3: Reducción de dimensionalidad utilizando T-SNE
tsne = TSNE(n_components=2)

# Configura el pipeline con los pasos definidos
pipeline = Pipeline(steps=[step1,
                            ('preprocessor', preprocessor), 
                            ('tsne', tsne)])

# Aplica el pipeline al DataFrame df_retail
transformed_data = pipeline.fit_transform(df_retail)
df_transformed = pd.DataFrame(transformed_data, columns=['Componente 1', 'Componente 2'])



# Visualización de las componentes utilizando Plotly Express
fig = px.scatter(df_transformed, x='Componente 1', y='Componente 2', title='Visualización de T-SNE')
fig.show()


### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [114]:
k_values = list(range(1, 21))
inertia_values = []


for k in k_values:
    kmeans = KMeans(n_clusters=k,n_init='auto')
    pipeline_codo = Pipeline(steps=[step1, ('preprocessor', preprocessor), ('kmeans', kmeans)])
    pipeline_codo.fit(df_retail)  # Reemplaza df_retail por tus datos
    
    inertia_values.append(pipeline_codo.named_steps['kmeans'].inertia_)

# Graficar el método del codo
fig = go.Figure()
fig.add_trace(go.Scatter(x=k_values, y=inertia_values, mode='lines+markers'))
fig.update_layout(
    xaxis_title='Número de clusters (k)',
    yaxis_title='Inertia',
    title='Método del codo para K-means'
)
fig.show()

Después de analizar el gráfico resultante del método del codo, se observa que la inercia disminuye significativamente a medida que el número de clusters aumenta hasta aproximadamente 4 clusters, momento en el cual la disminución se vuelve menos pronunciada. Por lo tanto, se selecciona 4 como el número óptimo de clusters para el modelo de K-means en este caso.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [115]:
k = 4
# Entrenar el modelo de K-means con el valor de k seleccionado
kmeans_final = KMeans(n_clusters=k, n_init='auto')
pipeline_kmeans = Pipeline(steps=[step1, ('preprocessor', preprocessor), ('kmeans', kmeans_final)])
pipeline_kmeans.fit(df_retail)  # Reemplaza df_retail por tus datos

Pipeline(steps=[('custom_features',
                 FunctionTransformer(func=<function custom_features at 0x0000019C83627920>)),
                ('preprocessor',
                 ColumnTransformer(transformers=[('minmax', MinMax(),
                                                  ['Length', 'Recency',
                                                   'Monetary', 'Frequency',
                                                   'Periodicity'])])),
                ('kmeans', KMeans(n_clusters=4, n_init='auto'))])

In [116]:
# Obtener las etiquetas de los clústeres asignados a cada muestra
labels = pipeline_kmeans.named_steps['kmeans'].labels_

# Agregar las etiquetas de clústeres como una nueva columna en el DataFrame original
df_features['Cluster'] = labels

# Calcular los promedios para cada atributo, agrupando por el clúster
cluster_means = df_features.groupby('Cluster').mean()

# Mostrar la tabla de promedios por clúster
cluster_means

C:\Users\luish\AppData\Local\Temp\ipykernel_17128\3542472462.py:8: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,Length,Recency,Monetary,Frequency,Periodicity
Cluster,,,,,
0,17.361538,4821.335165,54.875497,26.423077,2.737762
1,181.287139,4634.187438,27.344251,88.877368,20.912601
2,318.619217,4589.759786,35.539109,210.011566,20.118501
3,17.262334,4617.875489,33.087491,40.489428,2.394149


Al observar los resultados de la clusterización y las medias de los atributos LRMFP, se pueden identificar agrupaciones coherentes en función de las características analizadas.

El retail parece tener diferentes tipos de clientes basados en las siguientes características:

-**Length (L):** Los clientes del cluster 2 tienen la longitud promedio más alta, lo que indica que han estado interactuando con el retail durante un período más largo en comparación con los clientes de otros clusters.

-**Recency (R):** Los clientes del cluster 0 y 1 tienen valores de recencia significativamente más altos que los clientes de los otros clusters. Esto indica que han pasado más tiempo desde su última interacción con el retail, lo que podría sugerir una menor frecuencia de compra o menor lealtad.

-**Monetary (M):** Los clientes del cluster 0 tienen el valor promedio más alto en términos monetarios, lo que indica que gastan más en promedio durante cada visita al retail.

-**Frequency (F):** Los clientes del cluster 2 tienen la frecuencia promedio más alta, lo que significa que realizan más visitas y compras en el retail en comparación con los clientes de otros clusters.

-**Periodicity (P):** Los clientes del cluster 0 y 3 tienen valores de periodicidad más bajos, lo que sugiere que visitan el retail con mayor regularidad y tienen una mayor constancia en su comportamiento de compra.

En general, se puede inferir que el retail tiene diferentes segmentos de clientes, cada uno con características distintivas en términos de su interacción y comportamiento de compra.

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [117]:
# Crear un nuevo DataFrame con las proyecciones y las etiquetas de clústeres
df_transformed['Cluster'] = labels

# Graficar las proyecciones T-SNE con las etiquetas de clústeres como color
fig = px.scatter(df_transformed, x='Componente 1', y='Componente 2', color='Cluster',
                 title='Proyecciones T-SNE con etiquetas de clústeres')
fig.show()



La visualización obtenida a través de la clusterización presenta una representación adecuada de los grupos de clientes. Sin embargo, es importante destacar que debido a la irregularidad en la forma de los grupos, no se logran obtener clusters perfectamente definidos. Esta irregularidad puede deberse a diversos factores, como la variabilidad en los comportamientos de compra, la presencia de subgrupos dentro de los clusters o la existencia de outliers que no siguen patrones claros.

 Es importante considerar que existen algoritmos de clustering, como DBSCAN, que podrían mejorar significativamente los resultados. DBSCAN puede detectar agrupaciones de forma más efectiva en comparación con K-means, especialmente cuando los grupos presentan formas y tamaños irregulares. Por lo tanto, en el contexto de la clusterización de clientes del retail, DBSCAN podría mejorar la separación entre los grupos y permitir una identificación más precisa de los segmentos de clientes.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>